# Available ECMWF data

The lab has access to a set of ECMWF data downloaded at the \\139.191.244.110\h\ECMWH\ server. These data are stored in 2 folders

## WMO essentials

The first is the 'bufr' folder which stores for every year starting in 2013 a number of grib files with global data in 0.5 min resolution for a 'msl' (mean surface level) pressure, 'u' & 'v' veloctities, temperature 't' and geopotential height 'gh' at 2 isobaric levels (corresponding to different heights ?) namely 500 hPa and 850 hPa including the forecast for 240 h (10 days).

### Note: The resolution of the local data is 0.5 min while the data downloaded from the ECMWF url (see below) have 2.5 min resolution based on the same 0.5 resolution results.

Listing the files for the current day gives

In [ ]:
# Note that the above location is mounted on this system at "/mnt/ECMWF/"

path='/mnt/ECMWF/'

import datetime # for defining the date

date=datetime.datetime.today() # get today

#date=datetime.datetime.strptime('20160101.00','%Y%m%d.%H') # or fix a time stamp

dstr=date.strftime(format='%Y%m%d') # get it in string as it it appear on the file names

In [ ]:
# list the files for today

import os # module for os functions

[ f for f in os.listdir(path+'bufr/{}'.format(date.year)) if '{}'.format(dstr) in f ] # list only the files with matching date

As you can see above in the same directory there are also bufr files with the tracking of current Tropical cyclones (if any) with their name in the filename as descriptor. 

The list above (used to ) include the ensemble standart deviation ('es') forecast. We can mask them like... 

In [ ]:
all=[ f for f in os.listdir(path+'bufr/{}'.format(date.year)) if '{}'.format(dstr) in f ] # list only the files with matching date
[f  for f in all if '{}'.format('_es_') in f]

We can get the name of the variable by reading the data, e.g

In [ ]:
import pygrib # module to read grib files

for f in all:
    if '{}'.format('_es_') in f:
        data=pygrib.open(path+'bufr/{}/'.format(date.year)+f)
        print f
        print data.message(1)
        break

The ensemble mean ..

In [ ]:
[f  for f in all if '{}'.format('_em_') in f] # if any

In [ ]:
for f in all:
    if '{}'.format('_em_') in f:
        data=pygrib.open(path+'bufr/{}/'.format(date.year)+f)
        print f
        print data.message(1)
        break

The analysis data 'an' which correspond to the WMO essentials found in this webpage http://www.ecmwf.int/en/forecasts/datasets/wmo-essential-based-hres. 

In [ ]:
[f  for f in all if '{}'.format('_an_') in f]

In [ ]:
for f in all:
    if '{}'.format('_an_') in f:
        data=pygrib.open(path+'bufr/{}/'.format(date.year)+f)
        print f
        print data.message(1)


Note that the reference regular_ll means that the gridtype is 'regular_ll'=Regular longitude/latitude grid (cyl).

We can inspect the resolution as follows


In [ ]:
val=data[1]
val.data()
val.projparams

The later arrays are the latlons of the data. These data can be visualized like this [ECMWFvis](ECMWFvis.ipynb) 

## Tropical Storms 

A description for analysing and displaying the information regarding tropical storms can be found here [HECMWF](HECMWF.ipynb)

#### High resolution data

The second location is the folder 'grib' where we can get daily data in nested folders starting in 2015.

In [ ]:
path0=path+'grib/{}/{:02d}/{:02d}'.format(date.year,date.month,date.day-1) # we use yesterday for accessing all products
lst=os.listdir(path0)
lst

Find the index of the tropical_cyclone file in the list above

In [ ]:
xf=[x for x in lst if 'tropical' in x]
ix=lst.index(xf[0])
ix

The tropical_cyclone file contains

In [ ]:
dl=pygrib.open(path0+'/'+lst[ix])

In [ ]:
dl.messages # number of arrays included


In [ ]:
for g in dl:
    print g

In [ ]:
dl.rewind()

These data include the forecast for 72 hours (every hour) for pressure and U & V velocities at 10 m height. We can scan for e.g. pressure (references) to find...

In [ ]:
for g in dl:
   if 'pressure' in g.name: print g

The resolution is

In [ ]:
d=dl[1]
d.data()

In [ ]:
d.projparams
import numpy as np
np.shape(d.latlons())

The grid type before March 9, 2016 used to be 'reduced_gg' = Reduced Gaussian longitude/latitude grid  (https://en.wikipedia.org/wiki/Gaussian_grid). Note that although the data are in reduced_gg format the pygrib module expands the data in a regular grid 
(http://old.ecmwf.int/publications/manuals/libraries/interpolation/n640FIS.html).

It can be seen that the resolution was ~ 0.14 min. 

After March 9, 2016 the grid is regular_gg with grid size 2560x5120 and resolution ~ 7.5 km. (https://software.ecmwf.int/wiki/display/FCST/Introducing+the+octahedral+reduced+Gaussian+grid)

The other file in the folder is the 'boundary.grib' file. In this file there are the forecast for 24 hours of the following variables

In [ ]:
xf=[x for x in lst if 'boundary' in x]
ix=lst.index(xf[0])


bd=pygrib.open(path0+'/'+lst[ix])
var=[]
for g in bd:
    if not g.name in var: var.append(g.name)
        
var

One can find more info by listing any of them e.g...

In [ ]:
bd.rewind()
for g in bd:
    if var[20] in g.name: print g

Finally the 3rd file is the Runoff (https://software.ecmwf.int/wiki/display/S2S/S2S+Surface+Water+Runoff) data in netcdf format. To access these we can use the tarfile module

In [ ]:
xf=[x for x in lst if 'Runoff' in x]
ix=lst.index(xf[0])


In [ ]:

import tarfile
tlst=[]
tar=tarfile.open(path0+'/'+lst[ix], 'r:gz')
for f in tar:
    print f.name, f.size
    tlst.append(f.name)
tar.close()  

To see inside one of these netcdf files we need to extract it

In [ ]:
from netCDF4 import Dataset


In [ ]:
tar=tarfile.open(path0+'/'+lst[ix], 'r:gz')
for member in tar:
  f=tar.extract(member, path='tmp/')
  break
tar.close()

In [ ]:
dat=Dataset('tmp/{}'.format(tlst[0]))

In [ ]:
print dat

In [ ]:
print dat.variables

### Note that this files are written twice a day. One can list the files from previous day to verify this

In [ ]:
path1=path+'grib/'+str(date.year)+'/'+str(date.month).zfill(2)+'/'+str(date.day).zfill(2)
os.listdir(path1)
